This script downloads Sentinel-1 imagery from Google Earth Engine to Google Drive.

It assumes individual assets have been created on Google Earth Engine for site locations, although this is not necessary.

The script requires a .csv file of site locations, "new_site_data.csv".

Sentinel-1 .tif files are exported to Google Drive, to be downloaded and then processed using a guided filter. They are exported to the folders "s1_batch_A" (for Ascending) and "s1_batch_D" (for Descending).

In [ ]:
!pip install geemap


In [ ]:
import ee
import geemap
import os
import pandas as pd

In [ ]:
Map = geemap.Map()
Map

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
sites = pd.read_csv('/content/new_site_data.csv')
sites.head(5)

,Location,new_start,new_end,X,lon,lat,Pod1,Pod2,Pod3,Group,start_date,end_date,Ortho,Scene,order_id_s_h
0,choma_A000127,2017-09-02,2018-03-02,22,27.0922,-16.7795,A000127,NaN,NaN,choma,11/01/2017,05/31/2018,0,85,0d688599-b6bb-47d8-a3cd-a6a8d736ec5b
1,choma_A000127,2018-03-02,2018-07-30,22,27.0922,-16.7795,A000127,NaN,NaN,choma,11/01/2017,05/31/2018,0,85,0d688599-b6bb-47d8-a3cd-a6a8d736ec5b
2,choma_A000212,2017-09-02,2018-03-02,23,26.9253,-16.8294,A000212,NaN,NaN,choma,11/01/2017,05/31/2018,0,2,97cfdbe9-39d1-41ab-8e6e-3d355b598b85
3,choma_A000212,2018-03-02,2018-07-30,23,26.9253,-16.8294,A000212,NaN,NaN,choma,11/01/2017,05/31/2018,0,2,97cfdbe9-39d1-41ab-8e6e-3d355b598b85
4,choma_A000247,2017-09-02,2018-03-02,24,27.1179,-16.7724,A000247,NaN,NaN,choma,11/01/2017,05/31/2018,0,76,95c69caa-67fa-405f-ae2d-0456838c7272


Query and download Sentinel-1 files

In [ ]:
def toNatural(img):
  vv_nat = ee.Image(10.0).pow(img.select('VV').divide(10.0)).rename('VV')
  vh_nat = ee.Image(10.0).pow(img.select('VH').divide(10.0)).rename('VH')
  angle = img.select('angle')
  return vv_nat.addBands(vh_nat).addBands(angle).copyProperties(img)

def clip_roi(image, roi):
  return (image.clip(roi).copyProperties(image))


for k in range(len(sites)):
  current_site = sites.Location[k]
  print(current_site)
  print(k)
  long = sites.lon[k]
  lat = sites.lat[k]
  start_date = sites.new_start[k]
  end_date = sites.new_end[k]
  point_buffer = ee.Geometry.Point(long, lat).buffer(100).bounds()
  coll = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(start_date, end_date).filterBounds(point_buffer).map(toNatural).map(lambda image: clip_roi(image, point_buffer)).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))


    # download using geemap
  geemap.ee_export_image_collection(
      coll,
      out_dir = "/content/gdrive/MyDrive/s1_batch_A/" + current_site + "/",
      scale=10,
    #  crs=None,
      region=point_buffer,
      file_per_band=False,
      timeout=300,
      proxies=None
  )

  coll = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(start_date, end_date).filterBounds(point_buffer).map(toNatural).map(lambda image: clip_roi(image, point_buffer)).filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))


    # download using geemap
  geemap.ee_export_image_collection(
      coll,
      out_dir = "/content/gdrive/MyDrive/s1_batch_D/" + current_site + "/",
      scale=10,
    #  crs=None,
      region=point_buffer,
      file_per_band=False,
      timeout=300,
      proxies=None
  )


